In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/

Mounted at /content/drive
/content/drive/My Drive


In [3]:
data = loadmat('digit.mat')
train = data['X']
test = data['T']

print("Train data: {}".format(train.shape))
print("Test data:  {}".format(test.shape))

Train data: (256, 500, 10)
Test data:  (256, 200, 10)


In [ ]:
data = loadmat('digit.mat')
train = data['X']
test = data['T']

#データに対し，1と1以外で正解ラベルを付ける
print(train.shape)
print(train[:, 0:50, 0].shape)
#今は第三層目でどのクラスのデータかが分かれているが，これを一列のデータに整形する必要がある

print(np.concatenate([train[:, :, 0], train[:, :, 1], train[:, :, 2], train[:, :, 3], train[:, :, 4],
                      train[:, :, 5], train[:, :, 6], train[:, :, 7], train[:, :, 8], train[:, :, 9]], 1).shape)

train = np.concatenate([train[:, :, 0], train[:, :, 1], train[:, :, 2], train[:, :, 3], train[:, :, 4],
                                 train[:, :, 5], train[:, :, 6], train[:, :, 7], train[:, :, 8], train[:, :, 9]], 1)

y = np.concatenate([np.full(500, 1),np.full(4500, -1)])
#xは特徴量ベクトルになる
#今回は16*16=256個の特徴量があるとして考える？


#一旦，trainデータの数を各クラス50個*10クラスの500個にしてやってみる
data = loadmat('digit.mat')
train = data['X']
test = data['T']

train = np.concatenate([train[:, 0:50, 0], train[:, 0:50, 1], train[:, 0:50, 2], train[:, 0:50, 3], train[:, 0:50, 4],
                                 train[:, 0:50, 5], train[:, 0:50, 6], train[:, 0:50, 7], train[:, 0:50, 8], train[:, 0:50, 9]], 1)
y = np.concatenate([np.full(50, 1),np.full(450, -1)])
test = np.concatenate([test[:, 0:20, 0], test[:, 0:20, 1], test[:, 0:20, 2], test[:, 0:20, 3], test[:, 0:20, 4],
                                 test[:, 0:20, 5], test[:, 0:20, 6], test[:, 0:20, 7], test[:, 0:20, 8], test[:, 0:20, 9]], 1)
y_test = np.concatenate([np.full(20, 1),np.full(180, -1)])




#ガウスカーネル
def kernel(x1, x2, h):
  return np.exp(-np.linalg.norm(x1-x2)/(2*h**2))


#カーネルマトリクスの計算
k = np.zeros([train.shape[1], train.shape[1]])
for i in tqdm(range(train.shape[1])):
  for j in range(train.shape[1]):
    k[i,j] = kernel(train[:,i], train[:,j], 0.1)

(256, 500, 10)
(256, 50)
(256, 5000)


100%|██████████| 500/500 [00:10<00:00, 47.62it/s] 


(500, 500)
(500, 1)


In [4]:
from zmq import NULL
#ガウスカーネルを計算する関数
def kernel(x1, x2, h):
  return np.exp(-np.linalg.norm(x1-x2)/(2*h**2))


#クラスラベルを作成する関数
def create_class_label(class_id):
  if class_id == 0:
    return np.concatenate([np.full(500, 1),np.full(4500, -1)])
  elif class_id == 1:
    return np.concatenate([np.full(500, -1), np.full(500, 1),np.full(4000, -1)])
  elif class_id == 2:
    return np.concatenate([np.full(1000, -1), np.full(500, 1),np.full(3500, -1)])
  elif class_id == 3:
    return np.concatenate([np.full(1500, -1), np.full(500, 1),np.full(3000, -1)])
  elif class_id == 4:
    return np.concatenate([np.full(2000, -1), np.full(500, 1),np.full(2500, -1)])
  elif class_id == 5:
    return np.concatenate([np.full(2500, -1), np.full(500, 1),np.full(2000, -1)])
  elif class_id == 6:
    return np.concatenate([np.full(3000, -1), np.full(500, 1),np.full(1500, -1)])
  elif class_id == 7:
    return np.concatenate([np.full(3500, -1), np.full(500, 1),np.full(1000, -1)])
  elif class_id == 8:
    return np.concatenate([np.full(4000, -1), np.full(500, 1),np.full(500, -1)])
  elif class_id == 9:
    return np.concatenate([np.full(4500, -1), np.full(500, 1)])
  else:
    return NULL
    


#def one_vs_all(train, test):
h = 0.1 #ガウスカーネルのハイパーパラメータ
class_num = 10

#各クラスの訓練データ，テストデータをひとかたまりのデータに整形する
train = np.concatenate([train[:, :, 0], train[:, :, 1], train[:, :, 2], train[:, :, 3], train[:, :, 4],
                                train[:, :, 5], train[:, :, 6], train[:, :, 7], train[:, :, 8], train[:, :, 9]], 1)
test = np.concatenate([test[:, :, 0], test[:, :, 1], test[:, :, 2], test[:, :, 3], test[:, :, 4],
                                test[:, :, 5], test[:, :, 6], test[:, :, 7], test[:, :, 8], test[:, :, 9]], 1)

#カーネルマトリクスの計算
k = np.zeros([train.shape[1], train.shape[1]])
for i in tqdm(range(train.shape[1])):#可視化
  for j in range(train.shape[1]):
    k[i,j] = kernel(train[:,i], train[:,j], h)

#逆行列の計算(ここまでがすべての2クラス分類で共通)
inv_ktk = np.linalg.inv(k.T @ k)

#すべての識別関数の結果を格納しておく
f_all = np.zeros([class_num, test.shape[1]])

#回帰による2クラス分類を，クラス数分だけ繰り返す
for c_id in range(class_num):
  #正解ラベルの作成
  #y = np.concatenate([np.full(500, 1),np.full(4500, -1)]) #要変更
  y = create_class_label(c_id)

  #最小二乗問題を解く
  theta = inv_ktk @ k.T @ y[:, None]

  #判別関数の値を計算する
  #sigmas = []
  #for i in range(test.shape[1]):
  #  sigma=0
  #  for j in range(test.shape[1]):
  #    sigma += theta[j]*kernel(test[:,i], train[:,j],h)
  #  sigmas.append(sigma)

  #判別関数の値を計算する
  for i in range(test.shape[1]):
    sigma = 0
    for j in range(test.shape[1]):
      sigma += theta[j] * kernel(test[:,i], train[:,j], h)
    f_all[c_id, i] = sigma


#クラス数分だけ繰り返した2クラス分類の結果のうち，最も確からしいクラスに分類する
class_label = np.zeros(test.shape[1])

for i in range(test.shape[1]):
  class_label[i] = np.argmax(f_all[:,i])


#正解ラベルと比較し，正解率を求める
correct_class_label = np.concatenate([np.full(200, 0), np.full(200, 1), np.full(200, 2), np.full(200, 3), np.full(200, 4), np.full(200, 5), np.full(200, 6), np.full(200, 7), np.full(200, 8), np.full(200, 9)])

correct_class_label_num = 0
for i in range(test.shape[1]):
  if class_label[i] == correct_class_label[i]:
    correct_class_label_num += 1

accuracy = float(correct_class_label_num) / test.shape[1]
print(accuracy)

100%|██████████| 5000/5000 [03:31<00:00, 23.59it/s]


0.1


In [34]:
#正解ラベルの作成
  #y = np.concatenate([np.full(500, 1),np.full(4500, -1)]) #要変更
y = create_class_label(9)

  #最小二乗問題を解く
theta = inv_ktk @ k.T @ y[:, None]

  #判別関数の値を計算する
  #sigmas = []
  #for i in range(test.shape[1]):
  #  sigma=0
  #  for j in range(test.shape[1]):
  #    sigma += theta[j]*kernel(test[:,i], train[:,j],h)
  #  sigmas.append(sigma)

  #判別関数の値を計算する
sigmas = []
for i in range(test.shape[1]):
  sigma = 0
  for j in range(test.shape[1]):
    sigma += theta[j] * kernel(test[:,i], train[:,j], h)
  sigmas.append(sigma)

In [35]:
for i in range(test.shape[1]):
  print(sigmas[i])

[-1.78844197e-54]
[-2.38777428e-32]
[-3.93029973e-17]
[-4.71946341e-28]
[-1.62619609e-44]
[-8.84026823e-13]
[-5.57402699e-46]
[-2.52714011e-12]
[-4.8735299e-19]
[-1.55153133e-13]
[-1.00799245e-11]
[-6.90003209e-21]
[-6.48282227e-35]
[-7.9704027e-48]
[-1.79743584e-20]
[-3.66280038e-13]
[-9.95415272e-20]
[-3.48176053e-08]
[-1.68331676e-12]
[-5.56887021e-12]
[-3.18981486e-13]
[-5.32666956e-20]
[-6.97915193e-15]
[-1.22867894e-13]
[-1.43431053e-26]
[-1.32051607e-18]
[-1.62502537e-17]
[-4.16557996e-27]
[-5.09859617e-32]
[-1.27557804e-27]
[-6.38213364e-39]
[-1.45155879e-16]
[-1.8826316e-40]
[-4.96667744e-13]
[-5.02766278e-16]
[-9.13963943e-09]
[-2.75242876e-11]
[-3.37042417e-50]
[-1.43965812e-21]
[-2.81571701e-27]
[-1.28120397e-27]
[-7.74492334e-15]
[-9.0580972e-14]
[-2.22489154e-11]
[-1.46752939e-12]
[-4.82115986e-19]
[-2.35094914e-14]
[-1.52318392e-20]
[-3.11034906e-15]
[-7.16602228e-12]
[-5.15048714e-15]
[-6.58750649e-14]
[-1.46740251e-12]
[-2.36768069e-22]
[-4.11314781e-17]
[-2.32000257e-

In [ ]:
h = 0.1


# solve the least square problem
theta = np.linalg.inv(k.T @ k) @ k.T @ y[:, None]
print(theta.shape)

sigmas = []
for i in range(test.shape[1]):
  sigma=0
  for j in range(test.shape[1]):
    sigma += theta[j]*kernel(test[:,i], train[:,j],h)
  sigmas.append(sigma)

print(sigmas)


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-61-33e69a4a6fb2>", line 5, in <cell line: 5>
    theta = np.linalg.inv(k.T @ k) @ k.T @ y[:, None]
  File "<__array_function__ internals>", line 177, in inv
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb